In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center><font color= "green">PUBG Game Prediction</font></center>

<center><img src  ="https://media.giphy.com/media/zaUTLcbjhWpTrNYoFG/giphy.gif"></center>

<font color ="green">Importing Libraries<a class="anchor" id="import"></a>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"]=(16,6)
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import catboost as cb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


<font color ="green">Reading Data<a class="anchor" id="import"></a>

In [ ]:
df=pd.read_csv('/kaggle/input/pubg-games-dataset/PUBG_Game_Prediction_data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

<font color ="green">Data Wrangling <a class="anchor" id="import"></a>

**Check for the rows with missing win prediciton value**

In [ ]:
df[df['winPlacePerc'].isnull()]

In [ ]:
df.drop(2744604,inplace=True)

**Understanding Player Distribution in game**

In [ ]:
# Prepare new parameter to count how many new players are in game
df['PlayersJoined']=df.groupby('matchId')['matchId'].transform('count')    

In [ ]:
df.head()

In [ ]:
sns.countplot(df[df['PlayersJoined']>=75]['PlayersJoined'])
plt.show()

<font color ="green">Analysing data<a class="anchor" id="import"></a>

**Kills Without Moving ?**

<center><img src  ="https://i.gifer.com/g1OF.gif"></center>

In [ ]:
#prepare new data parameter to gather the total distance travelled
df['totalDistance']=df['rideDistance'] +df['swimDistance']+df['walkDistance'] 

#Preparing the data parameter to check for anmoly detection 
# that the person has not moved but still manages to do the kills

df['KillsWithoutMoving']=((df['kills']>0) & (df['totalDistance']==0))

In [ ]:
# Here we can see that there are
df[df['KillsWithoutMoving']==True].head()

In [ ]:
df[df['KillsWithoutMoving']==True].index

In [ ]:
# here 
df[df['KillsWithoutMoving']==True].shape

In [ ]:
#drop the table
df.drop(df[df['KillsWithoutMoving']==True].index,inplace=True)

**EXTRA ORDINARY ROAD KILLS ??**

<center><img src  ="https://media.giphy.com/media/rhaeywi71wkcXRTZK6/giphy.gif"></center>

In [ ]:
df[df['roadKills']>5].head()


In [ ]:
print(df[df['roadKills']>5].shape)

In [ ]:
#Dropping the Extraordinary road kills
df.drop(df[df['roadKills']>5].index,inplace=True)

In [ ]:
# cleared all the extraordinary road kills
print(df[df['roadKills']>5].shape)

## SO Many KILLS -HOW ??

In [ ]:
df[df['kills']>20].head()

In [ ]:
sns.countplot(data=df,x=df['kills']).set_title("Distribution of KILLS by a player")
plt.xlabel("Number of Kills")
plt.ylabel("Count of players")
plt.show()

In [ ]:
df[df['kills']>15].shape

In [ ]:
# Upto 15 kills can ne normal 
sns.countplot(x=df[df['kills']>=15]["kills"]).set_title("Distribution of KILLS by a player")
plt.xlabel("Number of Kills")
plt.ylabel("Count of players")
plt.show()

## Removing Kills greater than 20 from dataset

In [ ]:
df[df['kills']>20].shape

In [ ]:
# Dropping all 417 vlaues from given dataset
df.drop(df[df['kills']>20].index,inplace =True)

In [ ]:
df[df['kills']>20].shape

## HeadShot

<center><img src  ="https://media.giphy.com/media/Y0mmEpprOh5R9GZA0J/giphy.gif"></center>


In [ ]:
df['Headshot_rate']=df['headshotKills']/df['kills']
df['Headshot_rate']=df['Headshot_rate'].fillna(0)

In [ ]:
sns.distplot(df['Headshot_rate'],bins=10).set_title("Histogram showing the distribuyion of headshot rate")
plt.ylabel("Count of Players")
plt.show()

In [ ]:
df[(df['Headshot_rate']==1)& (df['kills']>5)].shape

In [ ]:
df.drop(df[(df['Headshot_rate']==1)& (df['kills']>6)].index,inplace=True)

## Longest Shot

<center><img src  ="https://tenor.com/bluCZ.gif"></center>

In [ ]:
sns.distplot(df['longestKill'],bins=50).set_title("Histogram Shoeing Longest Kill Distribution")
plt.ylabel('Count of Players')
plt.show()

In [ ]:
df[df['longestKill']>500].shape

In [ ]:
df.drop(df[df['longestKill']>500].index,inplace=True)

In [ ]:
df[df['longestKill']>500].shape

## Weapon Change

In [ ]:
sns.distplot(df['weaponsAcquired'],bins=100).set_title("Weapon Distribution")
plt.show()

In [ ]:
# Dropping the weapons changed more than 50 times
df[df['weaponsAcquired']>15].shape

In [ ]:
df.drop(df[df['weaponsAcquired']>15].index,inplace=True)

In [ ]:
df[df['weaponsAcquired']>15].shape

## EDA


In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
###sns.heatmap(df.corr(),annot=True)
#pplt.show()

## Feature Engineering

In [ ]:
normalising_factor=(100 - df['PlayersJoined']/100)+1

In [ ]:
normalising_factor

In [ ]:
df['killsNorm']=df['kills'] * normalising_factor
df['damageDealtNorm']=df['damageDealt'] * normalising_factor
df['maxPlaceNorm']=df['maxPlace']* normalising_factor
df['matchDurationNorm']=df['matchDuration']* normalising_factor

df['travelDistance']=df['rideDistance'] +df['swimDistance']+df['walkDistance'] 
df['healsnboost']=df['heals'] + df['boosts']
df['assist']=df['assists'] + df['revives']


In [ ]:
df.columns

In [ ]:
data=df.drop(columns=['Id','groupId','matchId','assists','boosts','walkDistance','swimDistance','rideDistance','heals','revives','kills','damageDealt','matchDuration','maxPlace'])

In [ ]:
data.head()

<center><font color= "green">ML - Catboost Model</font></center>

## Handling Categorical Values

In [ ]:
x=data.drop(['winPlacePerc'],axis=1)
y=data['winPlacePerc']

In [ ]:
features=x.columns

In [ ]:
x=pd.get_dummies(x,columns=['matchType','KillsWithoutMoving'])

In [ ]:
x.head()

In [ ]:
x.shape

In [ ]:
features=x.columns

## Scaling The data

In [ ]:
sc=StandardScaler()
sc.fit(x)
x=pd.DataFrame(sc.transform(x))

In [ ]:
x.head()

## Splitting Data

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=0)

In [ ]:
print(xtrain.shape, ytrain.shape)
print(xtest.shape, ytest.shape)

##  CatBOOST MODEL

In [ ]:
import catboost as cb

In [ ]:
train_dataset=cb.Pool(xtrain,ytrain)
test_dataset=cb.Pool(xtest,ytest)

In [ ]:
model=cb.CatBoostRegressor(loss_function='RMSE')

## GRID SEARCH

In [ ]:
grid={'iterations':[100, 150],
      'learning_rate':[0.03,0.1],
      'depth':[2 ,4, 6, 8]}

model.grid_search(grid, train_dataset)

In [ ]:
feature_importance_df=pd.DataFrame()
feature_importance_df['features']=features
feature_importance_df['importance']=model.feature_importances_

feature_importance_df=feature_importance_df.sort_values(by=['importance'],ascending=False)

In [ ]:
feature_importance_df

In [ ]:
plt.bar(feature_importance_df.features,feature_importance_df.importance,color='turquoise')
plt.ylabel("CatBoost Feature Importance")
plt.xticks(rotation=90)
plt.show()

In [ ]:
pred=model.predict(xtest)


In [ ]:
rmse=np.sqrt(mean_squared_error(ytest,pred))
r2=r2_score(ytest,pred)

print("TEsting Performance")

print("RMSE: {:.2f}".format(rmse))
print("R2{:.2f}".format(r2))

# Rmse is 0.08 which says that our model prediction is only 8 % wrong  whhich is good
# R2 is 0.93 very much close to 1 
# Model performance is good and it is not overfirring

<center><img src  ="https://media.giphy.com/media/KB89dMAtH79VIvxNCW/giphy.gif"></center>